## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-03-06-39-35 +0000,nypost,Felon charged in 9-year-old’s train station de...,https://nypost.com/2025/09/03/us-news/chicago-...
1,2025-09-03-06-35-29 +0000,nyt,Wildfire Destroys Buildings in Gold Rush Town ...,https://www.nytimes.com/2025/09/03/us/wildfire...
2,2025-09-03-06-33-20 +0000,nyt,China’s Military Spectacle,https://www.nytimes.com/2025/09/03/world/asia/...
3,2025-09-03-06-23-23 +0000,bbc,"Floods kill 30 and submerge 1,400 villages in ...",https://www.bbc.com/news/articles/cvg4g95y7pjo...
4,2025-09-03-06-21-08 +0000,bbc,"'Trump's out, Xi's in': BBC correspondents rea...",https://www.bbc.com/news/articles/c7545p2px5no...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2440/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
29,trump,51
21,china,18
22,military,13
163,house,13
242,new,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
14,2025-09-03-05-07-00 +0000,nypost,"Trump accuses Putin, Kim Jong Un and Xi Jinpin...",https://nypost.com/2025/09/03/us-news/trump-ac...,153
4,2025-09-03-06-21-08 +0000,bbc,"'Trump's out, Xi's in': BBC correspondents rea...",https://www.bbc.com/news/articles/c7545p2px5no...,112
246,2025-09-02-14-16-14 +0000,latimes,Trump deployment of military troops to Los Ang...,https://www.latimes.com/california/story/2025-...,108
205,2025-09-02-17-48-32 +0000,bbc,Trump's use of National Guard in Los Angeles w...,https://www.bbc.com/news/articles/c5ylyd9lkkqo...,107
81,2025-09-02-23-36-33 +0000,cbc,U.S. military kills 11 in strike on alleged dr...,https://www.cbc.ca/news/world/trump-venezuela-...,97


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
14,153,2025-09-03-05-07-00 +0000,nypost,"Trump accuses Putin, Kim Jong Un and Xi Jinpin...",https://nypost.com/2025/09/03/us-news/trump-ac...
205,56,2025-09-02-17-48-32 +0000,bbc,Trump's use of National Guard in Los Angeles w...,https://www.bbc.com/news/articles/c5ylyd9lkkqo...
12,55,2025-09-03-05-23-07 +0000,nypost,Vice President JD Vance and second lady Usha w...,https://nypost.com/2025/09/03/us-news/vice-pre...
0,45,2025-09-03-06-39-35 +0000,nypost,Felon charged in 9-year-old’s train station de...,https://nypost.com/2025/09/03/us-news/chicago-...
240,43,2025-09-02-14-38-46 +0000,bbc,Ex-actor Zack Polanski's unusual path to becom...,https://www.bbc.com/news/articles/clyrev00lwno...
88,43,2025-09-02-23-16-19 +0000,nypost,Dramatic footage shows US strike that oblitera...,https://nypost.com/2025/09/02/world-news/foota...
316,39,2025-09-02-08-23-36 +0000,nypost,"Sudan landslide kills over 1,000 people and wi...",https://nypost.com/2025/09/02/world-news/sudan...
58,38,2025-09-03-01-09-45 +0000,nypost,Anti-ICE TikTok influencer films her own arres...,https://nypost.com/2025/09/02/us-news/anti-ice...
92,37,2025-09-02-23-08-35 +0000,nypost,House Oversight Committee releases more than 3...,https://nypost.com/2025/09/02/us-news/house-ov...
115,35,2025-09-02-21-29-07 +0000,nypost,Would-be assassin Ryan Wesley Routh challenges...,https://nypost.com/2025/09/02/us-news/would-be...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
